# Results May 2019

In [ ]:
import os
from pathlib import Path
import time
from datetime import datetime
import itertools

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns
from sklearn.model_selection import KFold, RepeatedStratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import model_from_json

In [ ]:
sns.set(style="whitegrid")
colors = sns.color_palette()
colorsp1 = colors
colorsp1.pop(0)
colorsp1.append(colors[0])

def format_plot(ax, plt):
    ax.set_ylabel('validation accuracy')
    plt.ylim(bottom=0.35, top=1.05)
    ax.yaxis.set_major_formatter(FuncFormatter('{0:.0%}'.format))
    ax.set_xlabel('Dataset')
    plt.tight_layout()
    
def print_stats(data, name):
    print(name,  ':')
    print('Quantiles:\n', data['val_acc'].quantile([0.25, 0.5, 0.75]) )
    print('Mean:', data['val_acc'].mean())
    
def print_means(data, names):
    print('Mean averages:')
    for d, n, in zip(data, names):
        print(d['val_acc'].mean(), n)

In [ ]:
def readucr(filename):
    data = np.loadtxt(Path(filename))
    Y = data[:,0]
    X = data[:,1:]
    return X, Y


def dataset_size(filename):
    if 'private_dog0' == filename or 'private_dog1' == filename or 'private_dog2' == filename:
        fdir = '../../data/private_data/private_events_dev' 
    elif 'private' in filename:
        fdir = '../../data/private_data/private_events_dev2' 
        
    _, y_train = readucr(fdir+'/'+filename+'/'+filename+'_TRAIN.txt')
    _, y_test = readucr(fdir+'/'+filename+'/'+filename+'_TEST.txt')
    return y_train.shape[0]+y_test.shape[0]


def end_test_dataset_size(filename):
    fdir = '../../data/private_data/private_events_dev2' 
    _, y_endtest = readucr(fdir+'/'+filename+'/'+filename+'_END_TEST.txt')
    return y_endtest.shape[0]

# Confirm performance of untuned DNNs : GunPoint

In [ ]:
print('Single train and test. Same train:test split as UCR TSC archive. Reporting validation accuracy and error.', '\n')

print('1-NN (1-nearest neighbor)')
print('0.9133333333333333')
print(1-0.9133333333333333, 'error\n')

file = '../../logs/2019-05-10T20:18/GunPoint/mlpwang_summary.csv'
data1 = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
print(file, '(MLP Wang)')
print('No. results:', data1.shape[0])
print(data1.iloc[0]['val_acc'].mean())
print(1-data1['val_acc'].mean(), 'error\n')

file = '../../logs/2019-03-31T18:07/GunPoint/devnet_summary.csv'
data2 = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
print(file, '(FCN Wang)')
print('No. results:', data2.shape[0])
print(data2['val_acc'].mean())
print(1-data2['val_acc'].mean(), 'error\n')

file = '../../logs/2019-03-29T15:29/GunPoint/devnet_summary.csv'
#file = '../../logs/2019-05-11T13:42/GunPoint/resnet_summary.csv'
data2 = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
print(file, '(ResNet Wang)')
print('No. results:', data2.shape[0])
print(data2['val_acc'].mean())
print(1-data2['val_acc'].mean(), 'error\n')


In [ ]:
# 10-fold cross validation
print('1-NN (1-nearest neighbor)')
print(0.9450000000000001)
print(0.043779751788545644, '\n')

file = '../../logs/2019-03-17T16:35/GunPoint/mlpwang_summary.csv'
data1 = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
name1 = 'GunPoint'
print(file, '(MLP Wang)')
print(data1['val_acc'].mean())
print(data1['val_acc'].std())
print('Number of folds', data1['val_acc'].count(), '\n')

file = '../../logs/2019-05-09T09:25/GunPoint/devnet_summary.csv'
data2 = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
print(file, '(FCN Wang)')
print(data2['val_acc'].mean())
print(data2['val_acc'].std())
print('Number of folds', data2['val_acc'].count(), '\n')

file = '../../logs/2019-03-18T17:32/GunPoint/resnet_summary.csv'
data2 = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
print(file, '(ResNet Wang)')
print(data2['val_acc'].mean())
print(data2['val_acc'].std())
print('Number of folds', data2['val_acc'].count(), '\n')

# Untuned models : all dogs data (balanced dataset)

In [ ]:
all_data = list()
all_names = list()
results = []

file = '../../logs/2019-05-10T16:36/private_balanced/nearestneighbours_summary.csv'
data = pd.read_csv(file, header=None, names=['val_acc'])
all_data.append(data['val_acc'])
all_names.append('1-NN')
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])

file = '../../logs/2019-03-21T14:23/private_balanced/mlpwang_summary.csv'
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
all_data.append(data['val_acc'])
all_names.append('MLP')
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])

file = '../../logs/2019-05-09T12:30/private_balanced/devnet_summary.csv'
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
all_data.append(data['val_acc'])
all_names.append('FCN')
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])

file = '../../logs/2019-03-20T19:47/private_balanced/resnet_summary.csv'
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
all_data.append(data['val_acc'])
all_names.append('ResNet')
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])

print(results)

ax = sns.boxplot(data=all_data)
ax = sns.swarmplot(data=all_data, color='black')
title = 'all_dogs (N='+str(dataset_size('private_balanced'))+')'
plt.suptitle(title)
plt.xticks([0, 1, 2, 3], [all_names[0], all_names[1], all_names[2], all_names[3]])
format_plot(ax, plt)
ax.set_xlabel('Model')
plt.ylim(bottom=0.30, top=1.05)
plt.savefig('boxplot_alldogs.png', bbox_inches='tight')


# Train on dog0 data only

In [ ]:
all_data = list()
all_names = list()
results = []

file = '../../logs/2019-05-10T10:19/private_dog0/nearestneighbours_summary.csv'
data = pd.read_csv(file, header=None, names=['val_acc'])
all_data.append(data['val_acc'])
all_names.append('1-NN')
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])

file = '../../logs/2019-03-17T14:56/private_dog0/mlpwang_summary.csv'
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
all_data.append(data['val_acc'])
all_names.append('MLP')
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])

file = '../../logs/2019-05-10T10:17/private_dog0/devnet_summary.csv'
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
all_data.append(data['val_acc'])
all_names.append('FCN')
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])

file = '../../logs/2019-03-18T19:12/private_dog0/resnet_summary.csv'
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
all_data.append(data['val_acc'])
all_names.append('ResNet')
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])

print(results)

ax = sns.boxplot(data=all_data)
ax = sns.swarmplot(data=all_data, color='black')
title = 'dog0 (N='+str(dataset_size('private_dog0'))+')'
plt.suptitle(title)
plt.xticks([0, 1, 2, 3], all_names)
format_plot(ax, plt)
ax.set_xlabel('Model')
plt.ylim(bottom=0.30, top=1.05)
plt.savefig('boxplot_allModels_dog0.png', bbox_inches='tight')

# Dog accuracy

In [ ]:
datadir = '../../data/private_data/private_events_dev2' 
fname = 'private_balanced'
meta_train = pd.read_csv(datadir+'/'+fname+'/'+fname+'_TRAIN_meta.txt', sep=',', parse_dates=['date'])
meta_test = pd.read_csv(datadir+'/'+fname+'/'+fname+'_TEST_meta.txt', sep=',', parse_dates=['date'])
meta = pd.concat([meta_train, meta_test])

colors = ['red', 'lightsalmon', 'palegreen', 'lime' ]   
meta.groupby('dog')['dog_result'] \
    .value_counts() \
    .sort_index(ascending=False) \
    .unstack(level=1) \
    .plot.bar(stacked=True, color=colors)

plt.xticks([0, 1, 2], ['0', '1', '2'])
plt.xticks(rotation=0)
plt.savefig('bar_dogAccuracy.png', bbox_inches='tight')

results = meta.pivot_table('run', index='dog', columns='dog_result', aggfunc=len, fill_value=0, margins=True)
print(results)


# Calculate Accuracy and FNR, false negative rate, etc.
P = results.TP+results.FN
N = results.TN+results.FP
results['Accuracy'] = (results.TP+results.TN)/(P+N)
results['TPR'] = results.TP/P
results['FPR'] = results.FP/N
results['TNR'] = results.TN/N
results['FNR'] = results.FN/P
print(results[['Accuracy', 'TPR', 'TNR', 'FPR', 'FNR']])

# MLP : all dogs - dog0 - dog0 correct

In [ ]:
all_data = list()
all_names = list()
results = []

file = '../../logs/2019-03-21T14:23/private_balanced/mlpwang_summary.csv'
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
all_data.append(data['val_acc'])
name = 'all_dogs (N='+str(dataset_size('private_balanced'))+')'
all_names.append(name)
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])

file = '../../logs/2019-03-17T14:56/private_dog0/mlpwang_summary.csv'
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
all_data.append(data['val_acc'])
name = 'dog0 (N='+str(dataset_size('private_dog0'))+')'
all_names.append(name)
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])

file = '../../logs/2019-03-17T18:16/private_dog0_correct/mlpwang_summary.csv'
#file = '../../logs/2019-05-11T08:53/private_dog0_correct_plus/mlpwang_summary.csv'
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
all_data.append(data['val_acc'])
name = 'dog0_correct (N='+str(dataset_size('private_dog0_correct'))+')'
all_names.append(name)
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])

print(results)

ax = sns.boxplot(data=all_data)
ax = sns.swarmplot(data=all_data, color='black')
plt.suptitle('MLP')
plt.xticks([0, 1, 2], [all_names[0], all_names[1], all_names[2]])
format_plot(ax, plt)
plt.savefig('boxplot_MLPWang.png', bbox_inches='tight')

# MLP : dog1 and dog2 all vs correct

In [ ]:
all_data = list()
all_names = list()
results = []
fig=plt.figure(figsize=(10, 4))

file = '../../logs/2019-03-17T14:56/private_dog0/mlpwang_summary.csv'
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
all_data.append(data['val_acc'])
name = 'dog0\n(N='+str(dataset_size('private_dog0'))+')'
all_names.append(name)
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])

file = '../../logs/2019-03-17T18:16/private_dog0_correct/mlpwang_summary.csv'
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
all_data.append(data['val_acc'])
name = 'dog0_correct\n(N='+str(dataset_size('private_dog0_correct'))+')'
all_names.append(name)
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])

file = '../../logs/2019-03-23T08:51/private_dog1/mlpwang_summary.csv'
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
all_data.append(data['val_acc'])
name = 'dog1\n(N='+str(dataset_size('private_dog1'))+')'
all_names.append(name)
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])

file = '../../logs/2019-03-24T09:56/private_dog1_correct/mlpwang_summary.csv'
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
all_data.append(data['val_acc'])
name = 'dog1_correct\n(N='+str(dataset_size('private_dog1_correct'))+')'
all_names.append(name)
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])

file = '../../logs/2019-03-23T22:02/private_dog2/mlpwang_summary.csv'
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
all_data.append(data['val_acc'])
name = 'dog2\n(N='+str(dataset_size('private_dog2'))+')'
all_names.append(name)
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])

file = '../../logs/2019-03-23T19:47/private_dog2_correct/mlpwang_summary.csv'
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
all_data.append(data['val_acc'])
name = 'dog2_correct\n(N='+str(dataset_size('private_dog2_correct'))+')'
all_names.append(name)
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])

print(results)

clrs = [colorsp1[0], colorsp1[1], colorsp1[0], colorsp1[1], colorsp1[0], colorsp1[1]]
ax = sns.boxplot(data=all_data, palette=clrs)
ax = sns.swarmplot(data=all_data, color='black')
plt.suptitle('MLP')
plt.xticks([0, 1, 2, 3, 4, 5], all_names)
format_plot(ax, plt)
plt.ylim(bottom=0.20, top=1.05)
plt.savefig('boxplot_MLPWang_dog012Correct.png', bbox_inches='tight')

# All models : dog0 correct

In [ ]:
all_data = list()
all_names = list()
results = []

file = '../../logs/2019-05-09T14:07/private_dog0_correct_plus/nearestneighbours_summary.csv'
data = pd.read_csv(file, header=None, names=['val_acc'])
all_data.append(data['val_acc'])
all_names.append('1-NN')
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])

file = '../../logs/2019-05-11T08:53/private_dog0_correct_plus/mlpwang_summary.csv'
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
all_data.append(data['val_acc'])
all_names.append('MLP')
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])

file = '../../logs/2019-05-09T15:15/private_dog0_correct_plus/devnet_summary.csv'
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
all_data.append(data['val_acc'])
all_names.append('FCN')
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])

file = '../../logs/2019-05-11T09:51/private_dog0_correct_plus/resnet_summary.csv'
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
print(data.head())
all_data.append(data['val_acc'])
all_names.append('ResNet')
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])

print('boxplot_allModels_dog0Correct')
print(results)

ax = sns.boxplot(data=all_data)
ax = sns.swarmplot(data=all_data, color='black')
title = 'dog0_correct (N='+str(dataset_size('private_dog0_correct_plus'))+')'
plt.suptitle(title)
plt.xticks([0, 1, 2, 3], [all_names[0], all_names[1], all_names[2], all_names[3]])
format_plot(ax, plt)
ax.set_xlabel('Model')
plt.ylim(bottom=0.30, top=1.05)
plt.savefig('boxplot_allModels_dog0Correct.png', bbox_inches='tight')

# MLP hyperparameter tuning

In [ ]:
# dog0 correct dataset
# 10-fold cross validation
acc = [[ 0.85,0.85, 0.85, 0.82, 0.79, 0.63, 0.53],
        [0.84,0.85, 0.84, 0.82, 0.86, 0.73, 0.56],
        [0.85,0.84, 0.85, 0.84, 0.85, 0.72, 0.62],
        [0.85,0.84, 0.84, 0.84, 0.8, 0.62, 0.52],
        [0.84,0.84, 0.84, 0.84, 0.84, 0.59, 0.5],
        [0.83,0.84, 0.84, 0.83, 0.84, 0.73, 0.5]]

std = [[ 0.045,0.05, 0.047, 0.051, 0.14, 0.16, 0.1],
        [0.044,0.042, 0.044, 0.041, 0.053, 0.14, 0.14],
        [0.039,0.051, 0.059, 0.049, 0.052, 0.17, 0.17],
        [0.053,0.044, 0.049, 0.042, 0.1, 0.18,  np.nan],
        [0.047,0.046, 0.046, 0.055, 0.039, 0.16,  np.nan],
        [0.042,0.046, 0.051, 0.05, 0.054, 0.17,  np.nan]]

fig = plt.figure(figsize=(10, 4))
ax0 = fig.add_subplot(121)
ax1 = fig.add_subplot(122, sharey=ax0)
sns.heatmap(acc, cmap='RdYlGn', annot=True, fmt = '.0%', cbar=False, ax=ax0)
sns.heatmap(std, cmap='RdYlGn_r', annot=True, fmt = '.1%', cbar=False, ax=ax1)
ax0.set_xlabel('Number of hidden layers')
ax1.set_xlabel('Number of hidden layers')
ax0.set_ylabel('Nodes per layer')
ax0.set_xticklabels([2, 3, 4, 5, 6, 7, 8])
ax1.set_xticklabels([2, 3, 4, 5, 6, 7, 8])
ax0.set_yticklabels([8, 16, 32, 64, 128, 256])
ax0.set_title('Validation accuracy')
ax1.set_title('Sample standard deviation of validation accuracy')
plt.savefig('heatmap_MLP_dog0Correct.png', bbox_inches='tight')

# Selected models

In [ ]:
all_data = list()
all_names = list()
results = []

file = '../../logs/2019-05-11T08:53/private_dog0_correct_plus/mlpwang_summary.csv' # 1h 16mins
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
all_data.append(data['val_acc'])
all_names.append('MLP\nWang et al.') #'1000/500/500/500/2')
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])

file = '../../logs/2019-05-12T14:20/private_dog0_correct_plus/devnet_summary.csv' # 3 mins
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time', 'end_test'])
all_data.append(data['val_acc'])
all_names.append('MLP\ntuned') #\n(1000/16/16/16/1)')
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])

file = '../../logs/2019-04-28T10:42/private_dog0_correct_plus/devnet_summary.csv' # 6 mins
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
all_data.append(data['val_acc'])
all_names.append('FCN\ntuned')
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])

file = '../../logs/2019-04-19T18:00/private_dog0_correct_plus/devnet_summary.csv' # 52 mins
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
all_data.append(data['val_acc'])
all_names.append('ResNet\ntuned')
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])

print(results)

ax = sns.boxplot(data=all_data)
ax = sns.swarmplot(data=all_data, color='black')
title = 'dog0_correct (N='+str(dataset_size('private_dog0_correct_plus'))+')'
plt.suptitle(title)
plt.xticks([0, 1, 2, 3], all_names)
format_plot(ax, plt)
ax.set_xlabel('DNN')
plt.ylim(bottom=0.30, top=1.05)
plt.savefig('boxplot_allTunedModels_dog0Correct.png', bbox_inches='tight')

# Dataset sizes

In [ ]:
def print_sizes(filename):
    if 'private_dog0' == filename or 'private_dog1' == filename or 'private_dog2' == filename:
        fdir = '../../data/private_data/private_events_dev' 
    elif 'private' in filename:
        fdir = '../../data/private_data/private_events_dev2' 
        
    _, y_train = readucr(fdir+'/'+filename+'/'+filename+'_TRAIN.txt')
    _, y_test = readucr(fdir+'/'+filename+'/'+filename+'_TEST.txt')
    if 'correct_plus' in filename:
        _, y_other = readucr(fdir+'/'+filename+'/'+filename+'_END_TEST.txt')
        meta = pd.read_csv(fdir+'/'+filename+'/'+filename+'_END_TEST'+'_meta.txt', sep=',', parse_dates=['date'])
    else:
        y_other = None
        
    print(filename)
    print('Train:', y_train.shape[0])
    print('Test:', y_test.shape[0])
    print('Train+Test:', y_train.shape[0]+y_test.shape[0])
    if y_other is not None:
        print('End test:', y_other.shape[0])
        print('class 0:', (y_other == 0).sum(), 'class 1:', (y_other == 1).sum())
        print_dog_accuracy(fdir, filename)
    print('\n')
    

def print_dog_accuracy(fdir, filename):
    meta = pd.read_csv(fdir+'/'+filename+'/'+filename+'_END_TEST'+'_meta.txt', sep=',', parse_dates=['date'])
    dog_correct = meta[meta['class']==meta['dog_pred']]
    dog_incorrect = meta[meta['class']!=meta['dog_pred']]
    print('dog correct:', dog_correct.shape[0], 'dog incorrect:', dog_incorrect.shape[0])
    print('dog accuracy:', dog_correct.shape[0]/(dog_correct.shape[0]+dog_incorrect.shape[0]))

In [ ]:
flist = ['private_balanced', 'private_correct_plus',
         'private_dog0', 'private_dog0_correct', 'private_dog0_correct_plus',
        'private_dog1', 'private_dog1_correct_plus',
        'private_dog2', 'private_dog2_correct_plus']
for fname in flist:
    print_sizes(fname)

In [ ]:
# result set sizes
file = '../../logs/2019-05-06T17:01/private_dog0_correct_plus/devnet_summary.csv' # 4 x 3-fold
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
print(file)
print(data.shape[0], '\n')

file = '../../logs/2019-05-12T14:20/private_dog0_correct_plus/devnet_summary.csv' # 1 x 10-fold
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
print(file)
print(data.shape[0], '\n')

# k-fold cross validation on end test
Train on balanced data where dog was correct. Test on balanced, realistic dataset (samples not used during training or validation).

4 iterations of 3-fold cross validation.

In [ ]:
all_data = list()
all_names = list()
results = []

file = '../../logs/2019-05-11T18:17/private_dog0_correct_plus/devnet_summary.csv' # 7 mins
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time', 'end_test_acc'])
all_data.append(data['end_test_acc'])
name = 'dog0\n(N='+str(end_test_dataset_size('private_dog0_correct_plus'))+')'
all_names.append(name) 
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])
print(file)
print('k-fold cv: N=', dataset_size('private_dog0_correct_plus'), 'val_acc', data['val_acc'].mean(), '\n')

file = '../../logs/2019-05-11T19:09/private_correct_plus/devnet_summary.csv' # 10 mins
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time', 'end_test_acc'])
all_data.append(data['end_test_acc'])
name = 'all_dogs\n(N='+str(end_test_dataset_size('private_correct_plus'))+')'
all_names.append(name) 
results.append([all_names[-1], all_data[-1].mean(), all_data[-1].std()])
print(file)
print('k-fold cv: N=', dataset_size('private_correct_plus'), 'val_acc', data['val_acc'].mean(), '\n')

print(results)

ax = sns.boxplot(data=all_data)
ax = sns.swarmplot(data=all_data, color='black')
plt.suptitle('MLP 1000/16/16/16/1') #('Evaluate MLP on realistic dataset')
plt.xticks([0, 1], all_names)
format_plot(ax, plt)
ax.set_xlabel('realistic, balanced dataset')
ax.set_ylabel('accuracy')
plt.ylim(bottom=0.55, top=0.75)
plt.savefig('MLP_tuned_realistic_data.png', bbox_inches='tight')